### The Mandelbrot Set Iteration

The Mandelbrot set is given as the iteration of the following equations
$$
z_{n+1} = z^2_n + c
$$

Where:
* $z_0$ = 0 (starting value)
* $c$ is a complex number, where each unique $c$ will yield a different sequence of $z$ values

A point $c$ is in the Mandelbort set if, after iterating the equation multiple times, $|z|$ (the magnitude of $Z$) stays bounded (specifically, it remains $\leq$ 2). If $|z|$ escapes beyond 2, $c$ is not part of the set

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from worker import worker_function
from worker import worker_pure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def mandelbrot(c_points, max_iter, escape_radius) -> tuple[np.array, np.array]:
    '''
    This function calculates the number of iterations until the magnitude of z escapes to infinity. 
    Within each iteration the z is updated with c, an imaginary number representing a grid point. 
    Ultimately, a mandelbrot is calculated. 
    '''
    iteration_count = np.zeros(c_points.shape)
    mandelbrot_set = np.zeros(c_points.shape, dtype=bool)
    for i in range(c_points.shape[0]):
        for j in range(c_points.shape[1]):

            # take a gridpoint
            c = c_points[i, j]
            z = 0
            for iteration in range(max_iter):

                # update of z
                z = z**2 + c
                if abs(z) > escape_radius:
                    mandelbrot_set[i, j] = False
                    iteration_count[i, j] =iteration
                    break
            else:
                mandelbrot_set[i, j] = True
                iteration_count[i, j] = max_iter
    return (mandelbrot_set, iteration_count)


            

#### Reading .txt file and processing data 

In [ ]:
import pandas as pd

df = pd.read_csv("question3-orthogonal.txt", header=1)  # Set header=None since there is no header in the file
print(df)
# Assign column labels
df.columns = ["grid_size", "max_iterations", "run", "total_points", "points_inside"]
df['fraction'] = 1- df['points_inside'] / df['total_points']

average_fraction_per_run = df.groupby('run')['fraction'].mean()
variance_fraction_per_run = df.groupby('run')['fraction'].var()
# Display the result
print(average_fraction_per_run)
print(variance_fraction_per_run)

#### Framework for Parallelized implementation -> for all sampling
Do not forget to add sampling specific code in the worker.py file, and add parameters underneath.   
Code is currently running with deterministic function mandelbrot in worker.py. Idea is to replace this function (do not remove mandelbrot tho) with sampling specific function.  
Also, in case of changes in worker.py, reload this module, such that notebook refreshes imports and notices changes.  

Copy this cell, to keep this framework reusable


In [ ]:
import os
from multiprocessing import Pool
# Import own fucntion from .py file -> write for own sampling method! parameters of this function = grid (already given), max iteration bound (already given), sampling specific parameters (need to be difined still)
# Don't forget to also return the parameter values (as specified in example worker function)
from worker import worker_function_sampling


def partition_func(pars):

    # max 10 processes
    PROCESSES = 10 
    dict_res = {}
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        results = pool.map(worker_function_sampling,  pars)
        for res in results:
            (tot_points, out_points), par = res
            dict_res[par] = (tot_points, out_points)
            print(f"Gridsize: {par[0]}, Iteration Bound:{par[1]}\nTotal points and points inside mandelbrot: {tot_points, out_points}")
    return dict_res

if __name__ == '__main__':
    # Specify the s_values and grid sizes you want to experiment with
    # add sampling specific parameters, which are used in the worker method. (such as number of random samples taken (suggestion: choose 3 values for this, low, med, high))
    # evaluate when stochastic method is better than deterministic one. 
    
    # These are the values to experiment with, test with dummy values (these are quicker)
    s_values = [50, 500, 2000]
    grids = [100, 1000, 5000]
    # sampling_specific = [x, y, z]

    # Dummy values
    # s_values = [10, 50, 60, 70]
    # grids = [10, 50, 100, 150]

    par_combos = []

    # every proc gets a full grid, every proc a different parameter set (defined above)
    # add an extra for loop for an extra parameter
    for g in grids:
        real = np.linspace(-2.0, 1.0, g)
        imag = np.linspace(-1.5, 1.5, g)
        real_grid, imag_grid = np.meshgrid(real, imag)
        c_points = real_grid + 1j * imag_grid
        for s in s_values:
            par_combos.append((c_points, g, s))
    saved_values = {}
    # make sure the number of procs you use does not exceed the processor count

    print(f"Number of available CPU cores: {os.cpu_count()}")
    for i in range(10):
        # provide random seed such that random function will be different
        random_seed = i+42
        par_combos = [(comb, random_seed) for comb in par_combos]
        saved_values = partition_func(par_combos)

        # ensure saving the values with this code:
        # CHANGE FILE NAME!!
        with open("question3-1.txt", "w") as file:
            # add parameter in case a parameter is added to the 
            if i == 0:
                file.write("grid_size, max_iterations, total_points, points_inside\n")
            file.write(f"run {i}:")
            for (key1, key2), (value1, value2)  in saved_values.items():
                file.write(f"{key1}, {key2}, {value1}, {value2}\n")


#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

#### Orthongonal Sampling

In [ ]:
import os
from multiprocessing import Pool
# Import own fucntion from .py file -> write for own sampling method! parameters of this function = grid (already given), max iteration bound (already given), sampling specific parameters (need to be difined still)
# Don't forget to also return the parameter values (as specified in example worker function)
import scipy.stats.qmc as sampling
from worker import worker_function_sampling

def orthogonal(gridsize, seed):
     orth_sampler = sampling.LatinHypercube(d=2, scramble=True, strength=2,seed=seed)
     orth_samples = orth_sampler.random(n=gridsize)
     return orth_samples

def partition_func(pars):

    # max 10 processes
    PROCESSES = 10 
    dict_res = {}
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        results = pool.map(worker_function_sampling,  pars)
        for res in results:
            (tot_points, out_points), par = res
            dict_res[par] = (tot_points, out_points)
            print(f"Gridsize: {par[0]}, Iteration Bound:{par[1]}, Run: {par[2]}\nTotal points and points inside mandelbrot: {tot_points, out_points}")
    return dict_res

if __name__ == '__main__':
    # Specify the s_values and grid sizes you want to experiment with
    # add sampling specific parameters, which are used in the worker method. (such as number of random samples taken (suggestion: choose 3 values for this, low, med, high))
    # evaluate when stochastic method is better than deterministic one. 
    
    # These are the values to experiment with, test with dummy values (these are quicker)
    # s_values = [50, 500, 2000]

    # grids need to be prime numbers for orthogonal to work
    # grids = [100, 500, 1000, 5000]
    primes_grids = [101, 503, 997, 4999]
    # sampling_specific = [x, y, z]

    # Dummy values
    s_values = [10, 50, 60, 70]
    primes_grids = [11, 53, 101, 149]
    par_combos = []
    
    # make sure the number of procs you use does not exceed the processor count

    print(f"Number of available CPU cores: {os.cpu_count()}")
    
    # 10 runs
    for i in range(10):
        # provide random seed such that random function will be different
        random_seed = i+42

        # every proc gets a full grid, every proc a different parameter set (defined above)
        # add an extra for loop for an extra parameter
        for g in primes_grids:
            
            # creating the samples
            points = orthogonal(g**2, random_seed)
            
            min_x, max_x = -2, 1
            min_y, max_y = -1.5, 1.5

            real = min_x + points[:, 0]*3
            imag = min_y + points[:, 1]*3

            c_points = real + 1j * imag

            # print(c_points)
            for s in s_values:
                par_combos.append((c_points, g, s, i))

    saved_values = {}
    saved_values = partition_func(par_combos)

    # ensure saving the values with this code:
    # CHANGE FILE NAME!!
    with open("question3-orthogonal.txt", "w") as file:
        # add parameter in case a parameter is added to the 
        file.write("grid_size, max_iterations, run, total_points, points_inside\n")
        for (key1, key2, run), (value1, value2)  in saved_values.items():
            file.write(f"{key1}, {key2}, {run}, {value1}, {value2}\n")


#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

In [ ]:
# grid_size = np.linspace(4, )
import os
from multiprocessing import Pool
from worker import worker_function

def partition_grid(grid, proc):
    grid_size = grid//proc
    rest = grid % proc

    indexes =[]
    index = 0

    real = np.linspace(-2.0, 1.0, grid)
    imag = np.linspace(-1.5, 1.5, grid)

    # Create a 2D grid of complex numbers c
    real_grid, imag_grid = np.meshgrid(real, imag)
    c_points = real_grid + 1j * imag_grid

    for _ in range(proc):
        if rest > 0:
            addit = 1
            rest -=1
        else: 
            addit = 0
        indexes.append((index, index+grid_size+addit))
        index += grid_size + addit
    
    c_slices = [c_points[start:end] for start, end in indexes]
    return c_slices


def driver_func(grid, s):

    # max 10 processes
    PROCESSES = 10 
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        pid = os.getpid()
        # print(f"Current Process ID: {pid}")

        # every process gets a fraction of the grid (row-wise partitioning)
        c_parts = partition_grid(grid, PROCESSES)

        args = [(c_slice, s) for c_slice in c_parts]
        results = pool.map(worker_function,  args)
        total_points_parts, inside_points_parts = zip(*results)
        total_points = sum(total_points_parts)
        inside_points= sum(inside_points_parts)
        print(f"Result: {results}\nTotal points and points inside mandelbrot: {total_points, inside_points}")
    return total_points, inside_points

if __name__ == '__main__':
    s_values = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]
    grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
    
    saved_values = {}
    # make sure the number of procs you use does not exceed the processor count
    print(f"Number of available CPU cores: {os.cpu_count()}")
    for grid in grids:
        for s_value in s_values:
            key = (grid, s_value)
            tot_points, in_points = driver_func(grid, s_value)
            saved_values[key] = (tot_points, in_points)

#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

In [ ]:
with open("question2.txt", "w") as file:
    file.write("grid_size, max_iterations, total_points, points_inside\n")
    for (key1, key2), (value1, value2)  in saved_values.items():
        file.write(f"{key1}, {key2}, {value1}, {value2}\n")


#### Plotting the resutls from question 2

In [ ]:
import pandas as pd
df = pd.read_csv("question2.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])
df["fraction_mand"] = 1 - df["points_inside"]/df["total_points"]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
for max_iter, group in df.groupby("max_iterations"):
    ax1.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

# ax1.set_xlabel("Grid Size")
ax1.set_ylabel("Fraction of Points Inside Mandelbrot")
ax1.set_title("Full Size")
ax1.set_xlabel("Grid Size")
ax1.legend(title="Iteration Bound")
ax1.set_xlim(0, 5040)
ax1.grid(True)
# ax1.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax2.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

ax2.set_xlabel("Grid Size")
# ax2.set_ylabel("Fraction of Points Inside Mandelbrot")
ax2.set_title("Zoomed In (y-Axes)")
# ax2.legend(title="bound on number of iterations")
ax2.set_xlim(0, 5040)
ax2.grid(True)
ax2.set_ylim(0.165, 0.175)
# ax2.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax3.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

ax3.set_xlabel("Grid Size")
# ax3.set_ylabel("Fraction of Points Inside Mandelbrot")
ax3.set_title("Zoomed In (x-Axes)")
# 3x2.legend(title="bound on number of iterations")
ax3.set_xlim(0, 5040)
ax3.grid(True)
ax3.set_xlim(0, 500)
# ax2.show()


fig.suptitle("Fraction of Points Inside Mandelbrot vs Grid-Size")
plt.tight_layout()
plt.show()

## Random Sampling 

In [ ]:
def partition_random(pars):
    PROCESSES = 10
    dict_res = {}
    with Pool(PROCESSES) as pool:
        results = pool.map(worker_pure, pars)
        for res in results:
            (tot_points, in_points), par = res
            dict_res[par] = (tot_points, in_points)

            total_area = (1 - (-2)) * (1.5 - (-1.5))
            estimated_area = (in_points / tot_points) * total_area
            print(f"Samples: {par[0]}, Iterations: {par[1]}, Area: {estimated_area:.6f}")
    return dict_res

if __name__ == "__main__":
    grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
    max_iterations = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]

    par_combos = []
    for size in grids:
        dummy_points = np.zeros((1, 1))
        for max_iter in max_iterations:
            par_combos.append((dummy_points, size, max_iter))
    print(f"Number of available CPU cores: {os.cpu_count()}")
    saved_values = partition_random(par_combos)

    with open('monte_carlo_results.txt', 'w') as file:
        file.write('grid_size, max_iterations, total_points, points_inside\n')
        for (key1, key2), (value1, value2) in saved_values.items():
            file.write(f"{int(np.sqrt(key1))}, {key2}, {value1}, {value2}\n")


In [ ]:
import pandas as pd
df = pd.read_csv("question2.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])
total_area = (1 - (-2)) * (1.5 - (-1.5))  # Total area of the complex plane region
df["fraction_mand"] = 1 - (df["points_inside"] / df["total_points"])
df['estimated_area'] = df['fraction_mand'] * 9

real_area = 1.506484531722232

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), dpi=300)
for max_iter, group in df.groupby("max_iterations"):
    ax1.plot(group["grid_size"], group["estimated_area"], label=f"{max_iter}", marker= 'o')

# ax1.set_xlabel("Grid Size")
ax1.axhline(y=real_area, color='0', linestyle='--', 
            label='Real Area ≈ 1.506')
ax1.set_ylabel("Fraction of Points Inside Mandelbrot")
ax1.set_title("Full Size")
ax1.set_xlabel("Grid Size")
ax1.legend(title="Iteration Bound")
ax1.set_xlim(0, 5040)
ax1.grid(True)
# ax1.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax2.plot(group["grid_size"], group["estimated_area"], label=f"{max_iter}", marker= 'o')

ax2.axhline(y=real_area, color='0', linestyle='--')
ax2.set_xlabel("Grid Size")
# ax2.set_ylabel("Fraction of Points Inside Mandelbrot")
ax2.set_title("Zoomed In (y-Axes)")
# ax2.legend(title="bound on number of iterations")
ax2.set_xlim(0, 5040)
ax2.grid(True)
ax2.set_ylim(1.5, 1.51)
# ax2.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax3.plot(group["grid_size"], group["estimated_area"], label=f"{max_iter}", marker= 'o')

ax3.axhline(y=real_area, color='r', linestyle='--')
ax3.set_xlabel("Grid Size")
# ax3.set_ylabel("Fraction of Points Inside Mandelbrot")
ax3.set_title("Zoomed In (x-Axes)")
# 3x2.legend(title="bound on number of iterations")
ax3.set_xlim(0, 5040)
ax3.grid(True)
ax3.set_xlim(0, 500)
# ax2.show()


fig.suptitle("Fraction of Points Inside Mandelbrot vs Grid-Size")
plt.tight_layout()
plt.show()

In [ ]:
# Read the data
df = pd.read_csv("monte_carlo_results.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])

# Calculate estimated area
total_area = (1 - (-2)) * (1.5 - (-1.5))  # Total area of the complex plane region
df["estimated_area"] = (df["points_inside"] / df["total_points"]) * total_area

# Real Mandelbrot set area
real_area = 1.506484531722232

# Create three subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5), dpi=300)

# Full size plot
for max_iter, group in df.groupby("max_iterations"):
    ax1.plot(group["total_points"], group["estimated_area"], 
             label=f"{max_iter}", marker='o')

ax1.axhline(y=real_area, color='0', linestyle='--', label='Real Area ≈ 1.506')
ax1.set_xlabel("Number of Sample Points")
ax1.set_ylabel("Estimated Area")
ax1.set_title("Full Size")
ax1.legend(title="Iteration Bound")
ax1.set_xscale('log')  # Use log scale for better visualization
ax1.grid(True)

# Zoomed in y-axis
for max_iter, group in df.groupby("max_iterations"):
    ax2.plot(group["total_points"], group["estimated_area"], 
             label=f"{max_iter}", marker='o')

ax2.axhline(y=real_area, color='0', linestyle='--', label='Real Area ≈ 1.506')
ax2.set_xlabel("Number of Sample Points")
ax2.set_title("Zoomed In (y-Axes)")
ax2.set_xscale('log')
# You might need to adjust these values based on your results
ax2.set_ylim(1.5, 1.51)  # Adjust these values based on your data
ax2.grid(True)

# Zoomed in x-axis
for max_iter, group in df.groupby("max_iterations"):
    ax3.plot(group["total_points"], group["estimated_area"], 
             label=f"{max_iter}", marker='o')

ax3.axhline(y=real_area, color='0', linestyle='--', label='Real Area ≈ 1.506')
ax3.set_xlabel("Number of Sample Points")
ax3.set_title("Zoomed In (x-Axes)")
ax3.set_xscale('log')
ax3.set_xlim(4500*5000, 5000*5000)  # Adjust these values based on your data
ax3.set_ylim(1.5, 1.55) 
ax3.grid(True)

fig.suptitle("Estimated Mandelbrot Set Area vs Number of Sample Points")
plt.tight_layout()
plt.show()

#### Making a Mandelbrot figure

In [ ]:
# Setting up the imaginary point set
real = np.linspace(-2.0, 1.0, 1000)
imag = np.linspace(-1.5, 1.5, 1000)

# Create a 2D grid of complex numbers c
real_grid, imag_grid = np.meshgrid(real, imag)
c_points = real_grid + 1j * imag_grid

max_iter = 250
escape_radius = 2

mandel_set, iter_count = mandelbrot(c_points, max_iter, escape_radius)





plt.figure(figsize=(12, 8), dpi=300)
plt.imshow(iter_count, extent=(-2.0, 1.0, -1.5, 1.5), cmap='plasma', origin='lower')
plt.xlabel("Re(c)")
plt.ylabel("Im(c)")
plt.title("Mandelbrot Set")
plt.colorbar(label="In Mandelbrot Set")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import colorsys

def mandelbrot_iteration(c, max_iter, escape_radius):
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > escape_radius:
            return n
    return max_iter

def compute_mandelbrot(real, imag, max_iter, escape_radius):
    height, width = len(imag), len(real)
    iteration_count = np.zeros((height, width), dtype=np.int64)
    
    # Create a mesh grid for vectorized computation
    real_grid, imag_grid = np.meshgrid(real, imag)
    c = real_grid + 1j * imag_grid
    z = np.zeros_like(c, dtype=np.complex128)
    
    for i in range(max_iter):
        mask = np.abs(z) <= escape_radius
        z[mask] = z[mask]**2 + c[mask]
        iteration_count[mask] = i
    
    # Set points that never escaped to max_iter
    iteration_count[np.abs(z) <= escape_radius] = max_iter
    
    return iteration_count

def create_custom_colormap():
    # Create colors for our custom colormap
    colors = []
    for i in range(256):
        # Convert HSV to RGB, cycling through hues
        hue = i/256
        saturation = 1.0
        value = 1.0 if i > 10 else i/10.0  # Darker colors for low iteration counts
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(rgb)
    
    # Add black for the Mandelbrot set itself
    colors.append((0, 0, 0))
    
    return LinearSegmentedColormap.from_list('custom', colors)

def plot_mandelbrot(iteration_count, real_range, imag_range, max_iter, 
                   zoom_center=None, zoom_width=None, title="Enhanced Mandelbrot Set"):
    plt.figure(figsize=(15, 10), dpi=300)
    
    # Create the plot with a custom colormap
    custom_cmap = create_custom_colormap()
    
    # Normalize iteration counts for smoother color transitions
    smooth_iter = iteration_count + 1 - np.log(np.log(np.abs(iteration_count)))/np.log(2)
    smooth_iter[iteration_count == max_iter] = max_iter
    
    # Plot with enhanced aesthetics
    plt.imshow(smooth_iter, 
              extent=(real_range[0], real_range[-1], imag_range[0], imag_range[-1]),
              cmap=custom_cmap,
              origin='lower',
              aspect='equal')
    
    # Add gridlines
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Enhance labels and title
    plt.xlabel("Re(c)", fontsize=12)
    plt.ylabel("Im(c)", fontsize=12)
    plt.title(title, fontsize=14, pad=20)
    
    # Add colorbar with custom label
    cbar = plt.colorbar(label="Iteration Count", pad=0.02)
    cbar.ax.set_ylabel("Iteration Count", fontsize=10)
    
    # Add zoom box if specified
    if zoom_center and zoom_width:
        zoom_rect = plt.Rectangle(
            (zoom_center[0] - zoom_width/2, zoom_center[1] - zoom_width/2),
            zoom_width, zoom_width,
            fill=False, color='white', linestyle='--'
        )
        plt.gca().add_patch(zoom_rect)
    
    plt.tight_layout()
    return plt.gcf()

# Parameters for the main plot
# Reduced resolution for faster computation since we don't have Numba
real = np.linspace(-2.0, 1.0, 1000)  
imag = np.linspace(-1.5, 1.5, 1000)
max_iter = 1000
escape_radius = 2.0

# Compute the main Mandelbrot set
iteration_count = compute_mandelbrot(real, imag, max_iter, escape_radius)

# Create main plot
main_plot = plot_mandelbrot(iteration_count, real, imag, max_iter, 
                           title="The Mandelbrot Set")
plt.show()

# Create a zoomed plot of an interesting region
zoom_center = (-0.7435, 0.1314)
zoom_width = 0.002

# Calculate new ranges for zoom
zoom_real = np.linspace(zoom_center[0] - zoom_width/2, 
                       zoom_center[0] + zoom_width/2, 1000)
zoom_imag = np.linspace(zoom_center[1] - zoom_width/2, 
                       zoom_center[1] + zoom_width/2, 1000)

# Compute zoomed region
zoom_iteration_count = compute_mandelbrot(zoom_real, zoom_imag, max_iter, escape_radius)

# Create zoomed plot
zoom_plot = plot_mandelbrot(zoom_iteration_count, zoom_real, zoom_imag, max_iter,
                           title=f"Mandelbrot Set Zoom (width={zoom_width:.6f})")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.qmc import LatinHypercube

# Define s_values and grids (from exercise 2)
s_values = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]
grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
#s_values = [20, 200, 500]
#grids = [10, 50, 100]

def mandelbrot(c_points, max_iter, escape_radius) -> tuple[int, int]:
    '''
    This function calculates the number of iterations until the magnitude of z escapes to infinity. 
    Within each iteration the z is updated with c, an imaginary number representing a grid point. 
    Ultimately, a mandelbrot is calculated. 
    '''
    # iteration_count = np.zeros(c_points.shape)
    # mandelbrot_set = np.zeros(c_points.shape, dtype=bool)
    number_inside = 0
    total_numbers = 0
    for i in range(c_points.shape[0]):
        for j in range(c_points.shape[1]):
            total_numbers +=1
            # take a gridpoint
            c = c_points[i, j]
            z = 0
            for iteration in range(max_iter):
                # update of z
                z = z**2 + c
                if abs(z) > escape_radius:
                    number_inside+=1
                    # mandelbrot_set[i, j] = False
                    # iteration_count[i, j] =iteration
                    break
            # else:
                # mandelbrot_set[i, j] = True
                # iteration_count[i, j] = max_iter
    return (total_numbers, number_inside)

def latin_hypercube_sampling(s_values, max_iter=100, escape_radius=2, scramble=True, seed=None):

    # Initialize the LatinHypercube sampler for a 2D grid
    sampler = LatinHypercube(d=2, scramble=scramble, seed=seed)
    sample = sampler.random(n=s_values)

    real_range = np.linspace(-2.0, 1.0, grid)
    imag_range = np.linspace(-1.5, 1.5, grid)
    
    real_samples = real_range[0] + sample[:, 0] * (real_range[-1] - real_range[0])
    imag_samples = imag_range[0] + sample[:, 1] * (imag_range[-1] - imag_range[0])

    complex_samples_LHS = real_samples + 1j * imag_samples 

    return complex_samples_LHS

for s in s_values:
    for grid in grids:

        complex_samples_LHS = latin_hypercube_sampling(s_values=s, max_iter=100, escape_radius=2).reshape((s, 1))
        total_points, number_inside = mandelbrot(complex_samples_LHS, max_iter=100, escape_radius=2)

        #print(f"s_values: {s}, Grid size: {grid}x{grid}, Total points: {total_points}, Points inside: {number_inside}")
